<a href="https://colab.research.google.com/github/hadwin-357/ProteinMPNN_breakdown/blob/main/model_utils_function_4_gather_nodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#test model utils function
# graphic neuro network function
# gether edge
'''
inputs:  edges (B, N, N, C) # Batch, node, node, feature  # this is a fully connected, self looped GNN
neighbor_idx (B, N, K) # Batch, node, index for k-nearest neighbors

Return:
Based on N (node index), K(neighbor, index)to gather according features from edges (B, N, K, C)
'''

In [4]:
def gather_nodes(nodes, neighbor_idx):
    # Features [B,N,C] at Neighbor indices [B,N,K] => [B,N,K,C]
    # Flatten and expand indices per batch [B,N,K] => [B,NK] => [B,NK,C]
    neighbors_flat = neighbor_idx.view((neighbor_idx.shape[0], -1)) # [B,N,K] => [B,NK]
    neighbors_flat = neighbors_flat.unsqueeze(-1).expand(-1, -1, nodes.size(2))  #[B,NK] => [B,NK,C]
    # Gather and re-pack
    neighbor_features = torch.gather(nodes, 1, neighbors_flat)
    neighbor_features = neighbor_features.view(list(neighbor_idx.shape)[:3] + [-1])  # [B,N,K,]
    return neighbor_features

In [5]:
import torch
# Create random node features
nodes = torch.randn(2, 3, 2)  # Batch size = 2, Number of nodes = 3, Number of features = 2

# Create random neighbor indices
neighbor_idx = torch.randint(0, 3, (2, 3, 2))  # Batch size = 2, Number of nodes = 3, Number of neighbors = 2

In [6]:
neighbor_features = gather_nodes(nodes, neighbor_idx)

print("Shape of neighbor_features:", neighbor_features.shape)
print("neighbor_features tensor:")
print(neighbor_features)

Shape of neighbor_features: torch.Size([2, 3, 2, 2])
neighbor_features tensor:
tensor([[[[-0.6743,  0.3180],
          [ 0.4900,  0.2105]],

         [[-0.0015, -0.5057],
          [ 0.4900,  0.2105]],

         [[-0.6743,  0.3180],
          [-0.0015, -0.5057]]],


        [[[ 0.5504, -1.0023],
          [-0.5935, -0.2101]],

         [[-0.1746, -0.2861],
          [ 0.5504, -1.0023]],

         [[ 0.5504, -1.0023],
          [ 0.5504, -1.0023]]]])


In [9]:
#verfiy
neighbor_idx[0,0,:] # node 0's neighbor index


tensor([0, 2])

In [10]:
print(nodes[0,0,:]) # batch 0, node 0, node feature
print(nodes[0,2,:]) # batch 0, node 2, node feature

tensor([-0.6743,  0.3180])
tensor([0.4900, 0.2105])


In [11]:
print(neighbor_features[0,0,:,:]) # bath 0, node 0, neighbor's node features

tensor([[-0.6743,  0.3180],
        [ 0.4900,  0.2105]])
